# CS 506 BU ISE Project Solar Panel Regular Expressions

CS 506 Spring 2018

Team: 

Jason Lu (jasonlu6@bu.edu)

Hugh Qu (hughfang@bu.edu)

Zixin "Cindy" Ding (cindydxz@bu.edu)

**Project Mockup**: on github website 

In [1]:
# imports 
import json
import csv 
import pandas as pd
from pandas import DataFrame as Df

# clustering imports 
from sklearn.cluster import KMeans
from sklearn import mixture
import scipy.cluster.hierarchy as hierarchy
import sklearn.metrics as metrics
from sklearn.preprocessing import MinMaxScaler

# plotting imports 
import sklearn.cluster 
import matplotlib 
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

# Source Code:

In [2]:
# sources: 

# (json.dumps):
# https://stackoverflow.com/questions/19697846/how-to-convert-csv-file-to-multiline-json

# (avoid the UTF-8 encoding error):
# https://stackoverflow.com/questions/5552555/unicodedecodeerror-invalid-continuation-byte

# (TFDIF Vectorizer and Histogram Count Vectorizer for frequency of words in comments):
# https://stackoverflow.com/questions/36572221/how-to-find-ngram-frequency-of-a-column-in-a-pandas-dataframe

# creating the master database (create and read CSV)
# https://pythonspot.com/files-spreadsheets-csv/

# deal with error larger than field limit problem
# https://stackoverflow.com/questions/15063936/
# csv-error-field-larger-than-field-limit-131072

# avoid the "np.nan is an invalid document, expected byte or unicode string" Value Type error 
# https://stackoverflow.com/questions/39303912/
# tfidfvectorizer-in-scikit-learn-valueerror-np-nan-is-an-invalid-document

# counter object for the histogram frequency 
# https://stackoverflow.com/questions/22303554/words-frequency-using-pandas-and-matplotlib

# counting unique values in a list
# https://stackoverflow.com/questions/12282232/
# how-do-i-count-unique-values-inside-an-array-in-python

# plotting a frequency histogram
# https://stackoverflow.com/questions/45080698/
# make-frequency-histogram-from-list-with-tuple-elements

# selecting rows from dataframe using only a specified column value 
# (masking method)
# https://stackoverflow.com/questions/17071871/
# select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas

# filtering rows containing a string pattern / regex in pandas dataframe 
# https://stackoverflow.com/questions/27975069/
# how-to-filter-rows-containing-a-string-pattern-from-a-pandas-dataframe

# regular expression: extract a data type from string in pandas 
# https://stackoverflow.com/questions/35376387/extract-int-from-string-in-pandas

This code is used to parse the assessor CSV file and building permits CSV file in order to 
extract the solar capacity of each parcel in Boston based on index of each parcel building. It is required to answer one of our project questions: 

What communities are not benefitting from energy efficient systems? 

The steps to extracting
the solar capacity are as follows: 

1) We parse the assessor CSV file 

2) We parse the building permits CSV file 

3) Open the building permits csv file for solar panel text extraction

4) Get only the worktype of each csv file 

5) We then filter out only for 'SOL' worktype and print out the corresponding commments

6) Drop NaN values (for cleaning the data)

7) Get all of the comments into a list format 

8) We use regular expression to filter each comment from solar panel description

9) We then get only comments with kilowatts (which will contain information for solar capacity)

10) Saved data to a CSV file 

In [10]:
# import for dataframe and plots
import matplotlib.pyplot as plt 
import pandas as pd 
from pandas import DataFrame as Df

In [11]:
# filter out for only 'SOL' and print out the comments corresponding 
work = pd.read_csv('buildingpermits.csv',encoding='latin-1')
work 

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PermitNumber,WORKTYPE,PermitTypeDescr,DESCRIPTION,Comments,APPLICANT,DECLARED_VALUATION,TOTAL_FEES,ISSUED_DATE,EXPIRATION_DATE,...,OWNER,OCCUPANCYTYPE,sq_feet,ADDRESS,CITY,STATE,ZIP,Property_ID,Parcel_ID,Location
0,A100071,COB,Amendment to a Long Form,City of Boston,Change connector link layout from attached enc...,Renee Santeusanio,40000.0,429.0,11/4/11 11:04,5/4/12 0:00,...,CITY OF BOSTON,Comm,170.0,175 W Boundary RD,West Roxbury,MA,2132,17268.0,2.012032e+09,"(42.260750000, -71.149610000)"
1,A100137,EXTREN,Amendment to a Long Form,Renovations - Exterior,Landscaping/stonework - amending permit #2801/...,NaN,15000.0,206.0,1/3/13 14:13,7/3/13 0:00,...,MIARA SIMON,1-2FAM,0.0,14 William Jackson AV,Brighton,MA,2135,149852.0,2.204944e+09,"(42.344600000, -71.154050000)"
2,A100764,INTEXT,Amendment to a Long Form,Interior/Exterior Work,Interior and exterior work as indicated on dra...,Gregory Pomeroy,750000.0,7565.0,1/5/12 10:57,7/5/12 0:00,...,CARSON DAVID THOMAS TS,1-2FAM,3663.0,81 Chestnut ST,Boston,MA,2108,31492.0,5.023630e+08,"(42.356838000, -71.071231000)"
3,A100858,EXTDEM,Amendment to a Long Form,Demolition - Exterior,Landmark letter and permit submittal on existi...,Arthur Choo,0.0,20.0,8/21/12 12:27,2/21/13 0:00,...,SDS HOSPITALITY LLC,Comm,0.0,81 S Huntington AV,Jamaica Plain,MA,2199,76833.0,1.001624e+09,"(42.329910000, -71.111540000)"
4,A101019,INTEXT,Amendment to a Long Form,Interior/Exterior Work,Amendment to add bathroom bedroom 3rd floor,JOHN MAHONY,4000.0,75.0,11/25/11 11:20,5/25/12 0:00,...,TROTMAN GERALD A,1-2FAM,1950.0,2 Clarkson ST,Dorchester,MA,2125,33505.0,1.502933e+09,"(42.309550000, -71.065430000)"
5,A101208,EXTREN,Amendment to a Long Form,Renovations - Exterior,Amending alt77244/2011 - changes being made to...,Mike Zaisser,600.0,32.0,10/21/11 15:49,4/21/12 0:00,...,SMYTHE ALICE P,1-2FAM,0.0,21-23 Lee ST,Jamaica Plain,MA,2130,84830.0,1.103034e+09,"(42.307200000, -71.111910000)"
6,A101379,INTREN,Amendment to a Long Form,Renovations -Interior NSC,Amend Long Form Permit 10-0840 As Follows: Mod...,Stephen Weinig,200000.0,2092.0,7/11/12 13:29,1/11/13 0:00,...,DYER DENNIS,Mixed,0.0,244 Newbury ST,Boston,MA,2116,101456.0,5.031610e+08,"(42.349410000, -71.082400000)"
7,A101551,INTEXT,Amendment to a Long Form,Interior/Exterior Work,Finishing basement;as part of the first floor ...,Christine McMahon,50000.0,562.0,4/26/12 18:06,10/26/12 0:00,...,ONE-08 N STREET CONDOMINIUM,1-2FAM,680.0,108 N ST,South Boston,MA,2127,100043.0,6.042520e+08,"(42.334520000, -71.031083000)"
8,A101691,COB,Amendment to a Long Form,City of Boston,New Trash Compactor and Lift,Paul Hardiman Jr.,63444.0,669.0,11/15/11 12:01,5/15/12 0:00,...,BOSTON REDEVELOPMNT AUTH,Multi,0.0,1990 Columbus AV,Roxbury,MA,2119,35523.0,1.101630e+09,"(42.315320000, -71.097200000)"
9,A101698,OTHER,Amendment to a Long Form,Other,application to amend existing permit # ALT 767...,Garland Building Company,0.0,20.0,11/30/11 13:36,5/30/12 0:00,...,99 SUMMER OWNER LLC,Comm,1500.0,99 Summer St,Boston,MA,2110,132058.0,3.045760e+08,"(42.353490000, -71.058660000)"


In [12]:
# avoid reading in NaN values 
filter_work = work[work['WORKTYPE'].notnull()]
filter_work = filter_work[filter_work['Comments'].notnull()]
# filter_work.head(10)

In [13]:
# reading the columns with only 'SOL' from worktype and corresponding comments 
# print out the capacity of the solar panels / energy systems 

# getting only the worktype and comments 
string_mask = filter_work[['WORKTYPE','Comments']]
string_mask

# filtering the worktype for 'SOL' only 

# regex expression of str.contains()
solar_mask = filter_work[filter_work['WORKTYPE'].str.contains("SOL*")]
# solar_mask

In [14]:
# imports 
# regular expression import 
import re
# import pandas 
import pandas as pd 

In [15]:
# get all of the comments into a list format 
df = solar_mask['Comments']
# df

solar_filter = filter_work[filter_work['Comments'].str.contains("solar*")]
# solar_filter

In [16]:
# extraction step for all float values 
extract = solar_filter['Comments'].str.extract('(\d+)').astype(float)
# filter out the NaNs 
extract_filter = solar_filter['Comments'].dropna()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [17]:
import re

#get only comments with kilowatts 

# only decimal values for the kilowatts 
# drop NaN values and empty lists 
kw_extract = extract_filter.str.findall('\d+\.\d+').dropna()
kw_extract

109             [9.6]
414       [31.2, 7.5]
2617               []
2634               []
2667          [27.69]
2897           [67.2]
2901               []
2925               []
2944         [195.84]
3056               []
3066               []
3118               []
3145          [24.48]
3346               []
3365               []
3366               []
3375               []
3376               []
3377               []
3378               []
3379               []
3380               []
3381               []
3382               []
3383               []
3384               []
3385               []
3388               []
3389               []
3390               []
             ...     
354925             []
354939         [9.57]
355071         [4.02]
355087        [4.060]
355256         [6.38]
355258         [2.23]
355305         [3.77]
355355         [3.92]
355357         [6.44]
355373         [2.95]
355402        [11.21]
355408          [8.4]
355441        [3.245]
355458         [6.09]
355469    

In [18]:
frame = pd.Series.to_frame(kw_extract)
frame.to_csv("kw.csv")